# Pmax Helper - シンプル版（Google Colab完結）

LP URLを入力するだけで、AI動画を生成できます。フロントエンド・サーバー不要！

## 使い方
1. 上から順番にセルを実行
2. LP URLを入力
3. 「動画を生成」ボタンをクリック
4. 完成した動画をダウンロード

## 1. API キーの設定

左サイドバーの **🔑 鍵アイコン（Secrets）** から以下を追加:
- **FAL_KEY**: fal.ai APIキー (https://fal.ai/dashboard)
- **LAMBDA_SECRET_KEY**: ダミー値（例: `dummy_key`）

各シークレットの「ノートブックからアクセスを許可」をONにしてください。

In [ ]:
import os
from google.colab import userdata

# Google Colab Secretsから APIキーを取得
try:
    FAL_KEY = userdata.get('FAL_KEY')
    LAMBDA_SECRET_KEY = userdata.get('LAMBDA_SECRET_KEY')
    
    # 環境変数に設定
    os.environ['FAL_KEY'] = FAL_KEY
    os.environ['LAMBDA_SECRET_KEY'] = LAMBDA_SECRET_KEY
    
    print("✅ APIキーを設定しました")
except Exception as e:
    print("❌ エラー: Secretsが設定されていません")
    print("左サイドバーの🔑アイコンから FAL_KEY と LAMBDA_SECRET_KEY を追加してください")
    print(f"詳細: {e}")

## 2. セットアップ

In [ ]:
# リポジトリをクローン
!git clone https://github.com/KingHippopotamus/pmax-helper.git
%cd pmax-helper/backend

# 依存関係をインストール
!pip install -q flask flask-cors requests beautifulsoup4 Pillow fal-client python-dotenv ipywidgets

print("✅ セットアップ完了")

## 3. 動画生成ツール

下のセルを実行すると、入力フォームが表示されます。

In [ ]:
import sys
sys.path.insert(0, '/content/pmax-helper/backend')

from services.page_analyzer import PageAnalyzer
from services.video_generator import VideoGenerator
from services.scraper import ImageScraper
from services.image_processor import ImageProcessor
import ipywidgets as widgets
from IPython.display import display, HTML, Video
import requests

# 出力エリア
output = widgets.Output()

# UI要素
url_input = widgets.Text(
    placeholder='https://example.com',
    description='LP URL:',
    layout=widgets.Layout(width='600px')
)

analyze_button = widgets.Button(
    description='🔍 ページを分析',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

generate_button = widgets.Button(
    description='🎬 動画を生成',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

# グローバル変数
product_info = {}
character_image_url = ""
video_url = ""

def on_analyze_clicked(b):
    global product_info, character_image_url
    
    with output:
        output.clear_output()
        print("🔍 ページを分析しています...")
        
        try:
            # ページ分析
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            result = analyzer.analyze_page(url_input.value)
            
            if 'error' in result:
                print(f"❌ エラー: {result['error']}")
                return
            
            # 結果を保存
            product_info = result
            character_image_url = result.get('character_image_url', '')
            
            # 結果を表示
            output.clear_output()
            print("✅ 分析完了！\n")
            print(f"📊 商材名: {result.get('product_name', 'N/A')}")
            print(f"🎯 ターゲット: {result.get('target_audience', 'N/A')}")
            print(f"💡 キャッチコピー: {result.get('catchphrase', 'N/A')}")
            print(f"✨ ベネフィット1: {result.get('benefit1', 'N/A')}")
            print(f"✨ ベネフィット2: {result.get('benefit2', 'N/A')}")
            print(f"🎁 オファー: {result.get('offer', 'N/A')}")
            print(f"📢 CTA: {result.get('cta_text', 'N/A')}")
            print(f"\n🖼️ キャラクター画像: {character_image_url[:50]}..." if character_image_url else "")
            
            # 動画生成ボタンを有効化
            generate_button.disabled = False
            
        except Exception as e:
            output.clear_output()
            print(f"❌ エラーが発生しました: {str(e)}")

def on_generate_clicked(b):
    global video_url
    
    with output:
        output.clear_output()
        print("🎬 動画を生成しています...（数分かかる場合があります）")
        
        try:
            # キャラクター画像を取得
            if not character_image_url:
                print("❌ キャラクター画像が見つかりません")
                return
            
            # 画像をダウンロード
            response = requests.get(character_image_url, timeout=10)
            response.raise_for_status()
            image_data = response.content
            
            # 画像を前処理
            processor = ImageProcessor()
            processed_image = processor.preprocess_image(image_data)
            
            # プロンプトを生成
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            prompt = analyzer._generate_video_prompt(product_info)
            
            # 動画生成
            generator = VideoGenerator(FAL_KEY)
            result = generator.generate_character_video(processed_image, prompt=prompt)
            
            if 'error' in result:
                output.clear_output()
                print(f"❌ エラー: {result['error']}")
                return
            
            video_url = result.get('video_url', '')
            
            # 結果を表示
            output.clear_output()
            print("✅ 動画生成完了！\n")
            print(f"📹 動画URL: {video_url}")
            
            # 動画を表示
            display(HTML(f'<video width="640" controls><source src="{video_url}" type="video/mp4"></video>'))
            
            # ダウンロードリンク
            display(HTML(f'<a href="{video_url}" download="character_video.mp4" target="_blank">📥 動画をダウンロード</a>'))
            
        except Exception as e:
            output.clear_output()
            print(f"❌ エラーが発生しました: {str(e)}")
            import traceback
            traceback.print_exc()

# イベントハンドラを登録
analyze_button.on_click(on_analyze_clicked)
generate_button.on_click(on_generate_clicked)

# UIを表示
display(widgets.VBox([
    widgets.HTML("<h2>🎬 Pmax Helper - AI動画生成ツール</h2>"),
    url_input,
    widgets.HBox([analyze_button, generate_button]),
    output
]))

print("\n👆 上のフォームにLP URLを入力して、ボタンをクリックしてください")

---

## トラブルシューティング

### エラーが出る場合
- APIキーが正しく設定されているか確認
- LP URLが正しいか確認
- セルを上から順番に実行したか確認

### 動画生成に時間がかかる
- fal-ai APIは通常2-5分かかります
- セッションがタイムアウトした場合は最初から再実行